In [60]:
import numpy as np
import random

class QuantumParticle:
    def __init__(self, position, velocity):
        self.position = position
        self.velocity = velocity
        self.pbest_position = position
        self.pbest_fitness = -float("inf")
        self.fitness = -float("inf")

    def update_position(self, c1, c2, w, number_of_mecs, gbest_position):
        r1 = random.random()
        r2 = random.random()

        phi1 = (np.pi * r1) - (np.pi / 2)
        phi2 = (np.pi * r2) - (np.pi / 2)

        chi1 = np.cos(phi1)
        chi2 = np.cos(phi2)

        self.velocity = w * self.velocity + c1 * chi1 * (self.pbest_position - self.position) + c2 * chi2 * (gbest_position - self.position)
        self.position = int(np.clip(self.position + self.velocity, 0, number_of_mecs - 1))

    def calculate_fitness(self, transmission_rates, task_data, mec_computing_capacity):
        offloading_time = np.array(task_data) / transmission_rates[self.position]
        execution_time = np.array(task_data) / mec_computing_capacity
        total_delay = offloading_time + execution_time
        self.fitness = np.mean(1 / total_delay)

class QuantumParticleSwarmOptimization:
    def __init__(self, transmission_rates, number_of_devices, task_data, maximum_tolerable_delay, device_computing_capacity, number_of_mecs, mec_computing_capacity, transmission_power):
        self.transmission_rates = transmission_rates
        self.task_data = task_data
        self.number_of_mecs = number_of_mecs
        self.mec_computing_capacity = mec_computing_capacity

        self.particles = []
        for _ in range(len(number_of_devices)):
            position = np.random.randint(0, self.number_of_mecs)
            velocity = np.random.uniform(-1, 1)
            self.particles.append(QuantumParticle(position, velocity))

        self.gbest_position = self.particles[0].position
        self.gbest_fitness = -float("inf")

    def optimize(self, c1, c2, w, max_iterations):
        for iteration in range(max_iterations):
            for particle in self.particles:
                particle.update_position(c1, c2, w, self.number_of_mecs, self.gbest_position)
                particle.calculate_fitness(self.transmission_rates, self.task_data, self.mec_computing_capacity)

                if particle.fitness > particle.pbest_fitness:
                    particle.pbest_position = particle.position
                    particle.pbest_fitness = particle.fitness

                if particle.fitness > self.gbest_fitness:
                    self.gbest_position = particle.position
                    self.gbest_fitness = particle.fitness

        return self.gbest_position

# Set the parameters
transmission_rates = [500, 1000, 1500, 2000, 2500, 3000]
number_of_devices = [50, 100, 150, 200, 250, 300]
task_data = [5, 10, 15, 20, 25, 30, 35, 40, 45]
maximum_tolerable_delay = 0.5
device_computing_capacity = 0.5
number_of_mecs = 10
mec_computing_capacity = 4
transmission_power = 0.5

# Initialize the QPSO algorithm
qpso = QuantumParticleSwarmOptimization(transmission_rates, number_of_devices, task_data, maximum_tolerable_delay, device_computing_capacity, number_of_mecs, mec_computing_capacity, transmission_power)

# Optimize the task offloading strategy
gbest = qpso.optimize(c1=2, c2=2, w=0.7, max_iterations=100)

# Print the optimal task offloading strategy
print(gbest)



IndexError: list index out of range